In [1]:
import requests
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import gamma
import numpy as np

In [2]:
import c3aidatalake

###Todo (by 11/3):
- Quantify government policy categories (LocationPolicySummary)
    - Bigger values for more restrictive policies
- Convert economic data to usable format
- Produce/visualize distributions and general statistics for data
    - Visualize dates of latest policies for states


In [3]:
policies = c3aidatalake.fetch(
    "locationpolicysummary", 
    {
        "spec" : {
            "limit" : -1
        }
    })

Generating numerical columns from statePolicies:

Sora's Dictionaries:
'mandatoryQuarantine',
       'nonEssentialBusiness', 'largeGatherings', 'schoolClosure',
       'restaurantLimit', 'barClosures', 'faceCoveringRequirement',
       'PrimaryElectionPostponement', 'emergencyDeclaration',


In [4]:
{
    "tags": [
        "hide-input",
    ]
}
quantifyDicts = {}
quantifyDicts["easingOrder"] = {
    "Reopened" : 0, 
    "Proceeding with Reopening" : 1,
    "Paused" : 2, 
    "New Restrictions Imposed" : 3
}
quantifyDicts["stayAtHome"] = {
    "No Action" : 0,
    "Lifted" : 0,
    "Rolled Back to High Risk Groups" : 1,
    "New Stay at Home Order" : 2,
    "Statewide" : 2
}
quantifyDicts["mandatoryQuarantine"]  = {
    "No Action" : 0,
    "Lifted" : 0,
    "From Certain States (New)" : 1,
    "Rolled Back to Certain States" : 1,
    "From Certain States" : 1,
    "Rolled Back to International Travel" : 2,
    "All Travelers" : 3
}
quantifyDicts["nonEssentialBusiness"] = {
    "No Action" : 0,
    "All Non-Essential Businesses Permitted to Reopen" : 0,
    "Some Non-Essential Businesses Permitted to Reopen" : 1,
    "All Non-Essential Businesses Permitted to Reopen with Reduced Capacity" : 1,
    "Some Non-Essential Businesses Permitted to Reopen with Reduced Capacity" : 2,
    "New Business Closures or Limits" : 3
}
quantifyDicts["largeGatherings"] = {
    "Lifted" : 0,
    "No Action" : 0,
    "Expanded to New Limit Above 25" : 1,
    "New Limit on Large Gatherings in Place" : 1,
    "Expanded to New Limit of 25 or Fewer" : 2,
    ">10 People Prohibited" : 3,
    "All Gatherings Prohibited" : 4
}
quantifyDicts["schoolClosure"] = {
    "Rescinded" : 0,
    "Recommended Closure for School Year" : 1,
    "Recommended Closure" : 2,
    "Closed for School Year" : 3,
    "Closed" : 4
}
quantifyDicts["restaurantLimit"] = {
    "No Action" : 0,
    "Reopened to Dine-in Service" : 1,
    "Reopened to Dine-in Service with Capacity Limits" : 2,
    "New Service Limits" : 3,
    "Newly Closed to Dine-in Service" : 3
}
quantifyDicts["barClosures"] = {
    "Reopened" : 0,
    "New Service Limits" : 1,
    "Closed" : 2,
    "Newly Closed" : 2
}
quantifyDicts["faceCoveringRequirement"] = {
    "No" : 0,
    "Required for Certain Employees" : 1,
    "Allows Local Officals to Require for General Public" : 1,
    "Required for Certain Employees; Allows Local Officials to Require for General Public" : 1,
    "Required for General Public" : 2
}
quantifyDicts["PrimaryElectionPostponement"] = {
    "No" : 0,
    "Postponed" : 1,
    "Canceled" : 2
}
quantifyDicts["emergencyDeclaration"] = {
    "Yes" : 0
}
quantifyDicts["waiveTreatmentCost"]  = {
    "No Action" : 0,
    "State-Insurer Agreement" : 1,
    "State Requires" : 2
}  
quantifyDicts["freeVaccine"] = {
    "No Action" : 0,
    "State-Insurer Agreement" : 1,
    "State Requires" : 2
}
quantifyDicts["waiverOfPriorAuthorizationRequirements"] = {
    "No Action" : 0,
    "For COVID-19 Testing" : 1,
    "For COVID-19 Testing and Treatment" : 2
}
quantifyDicts["prescriptionRefill"] = {
    "No Action" : 0,
    "Expired" : 1,
    "State Requires" : 2
}
quantifyDicts["premiumPaymentGracePeriod"] = {
    "No Action" : 0,
    "Expired" : 1,
    "COVID-19 Diagnosis/Impacts Only" : 2,
    "Grace Period Extended for All Individual Policies" : 3,
    "All Policies" : 4
}
quantifyDicts["marketplaceSpecialEnrollmentPeriod"] = {
    "No" : 0,
    "Ended" : 1,
    "Active" : 2
}
quantifyDicts["section1135Waiver"] = {
    "Unapproved" : 0,
    "Approved" : 1
}
quantifyDicts["paidSickLeaves"] = {
    "No Action" : 0,
    "Proposed - March 2020" : 1,
    "Enacted" : 2
}
quantifyDicts["expandsAccesstoTelehealthServices"] = {
    "No" : 0,
    "Yes" : 1
}

In [5]:
statePolicies = policies.drop(44)

In [6]:
def mapperGenerator(colName):
    if colName in quantifyDicts.keys():
        def mapper(val):
            return quantifyDicts[colName][val]
    else:
        def mapper(val):
            return val
    return mapper

In [7]:
for col in statePolicies.columns:
    mper = mapperGenerator(col)
    statePolicies[col] = statePolicies[col].apply(mper)
statePolicies

,easingOrder,stayAtHome,mandatoryQuarantine,nonEssentialBusiness,largeGatherings,schoolClosure,restaurantLimit,barClosures,faceCoveringRequirement,PrimaryElectionPostponement,...,premiumPaymentGracePeriod,marketplaceSpecialEnrollmentPeriod,section1135Waiver,paidSickLeaves,expandsAccesstoTelehealthServices,id,lastSavedTimestamp,numSavedVersions,savedVersion,location.id
0,2,0,0,1,0,3,1,0,2,0,...,0,0,1,0,0,Alabama_UnitedStates_Policy,2020-09-12T02:52:54Z,6,-1,Alabama_USA
1,1,0,3,0,0,3,1,0,1,0,...,1,0,1,0,1,Alaska_UnitedStates_Policy,2020-09-12T02:52:54Z,5,-1,Alaska_UnitedStates
2,3,0,0,3,1,3,3,2,1,0,...,0,0,1,2,1,Arizona_UnitedStates_Policy,2020-09-12T02:52:54Z,5,-1,Arizona_UnitedStates
3,2,0,0,0,0,3,2,0,2,0,...,2,0,1,0,1,Arkansas_UnitedStates_Policy,2020-09-12T02:52:54Z,4,-1,Arkansas_UnitedStates
4,3,2,0,3,4,1,3,2,2,0,...,0,2,1,2,1,California_UnitedStates_Policy,2020-09-12T02:52:54Z,5,-1,California_UnitedStates
5,3,0,0,2,2,3,2,2,2,0,...,0,1,1,2,1,Colorado_UnitedStates_Policy,2020-09-12T02:52:54Z,5,-1,Colorado_UnitedStates
6,1,0,1,1,2,3,2,2,2,1,...,1,1,1,2,1,Connecticut_UnitedStates_Policy,2020-09-12T02:52:54Z,5,-1,Connecticut_UnitedStates
7,3,0,0,2,1,3,2,1,2,1,...,1,0,1,0,1,Delaware_UnitedStates_Policy,2020-09-12T02:52:54Z,5,-1,Delaware_UnitedStates
8,1,0,1,2,1,3,2,2,2,0,...,0,2,1,2,1,District of Columbia_UnitedStates_Policy,2020-09-12T02:52:54Z,5,-1,DistrictofColumbia_UnitedStates
9,3,0,0,1,0,1,2,2,1,0,...,0,0,1,0,0,Florida_UnitedStates_Policy,2020-09-12T02:52:54Z,5,-1,Florida_UnitedStates


In [12]:
stategdp = pd.read_excel(io = "qgdpstate1020_0.xlsx", index_col = 0, header = 1)
stategdp

,2019Q1,2019Q2,2019Q3,2019Q4,2020Q1,2020Q2
United States,2.9,1.5,2.6,2.4,-5.0,-31.4
New England,5.2,-0.1,1.6,1.0,-4.8,-32.3
Connecticut,4.9,-1.1,0.6,1.9,-6.0,-31.1
Maine,7.6,-0.4,4.6,3.4,-6.5,-34.4
Massachusetts,4.9,1.3,2.3,0.4,-4.3,-31.6
New Hampshire,8.2,-2.8,0.3,-0.5,-2.2,-36.9
Rhode Island,2.6,-1.6,-2.4,2.3,-5.2,-32.4
Vermont,3.8,-2.8,1.5,0.9,-5.8,-38.2
Mideast,3.5,1.8,1.0,1.6,-5.3,-34.0
Delaware,4.8,0.3,-0.4,1.7,-11.4,-21.9
